In [ ]:
'''
%pip install rioxarray
%pip install cartopy
'''

In [ ]:
import rioxarray as rxr
import xarray as xr
from pyproj import Transformer
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import glob
import numpy as np
from matplotlib.animation import FuncAnimation

In [ ]:
# set your own path names
remaMosaicPath = r"C:\Users\cambo\Desktop\ML-26\New folder (2)\MLGEO2026_Subglacial_Lakes\Notebooks\REMA\rema_mosaic_100m_v2.0_dem.tif"
cryosatPath = r"C:\Users\cambo\Desktop\ML-26\New folder (2)\MLGEO2026_Subglacial_Lakes\Notebooks\cryosat_downloads\CS_OFFL_THEM_GRID__ANTARCTIC_"
# ^^^ do not include a date or "_V201.nc" at the end, leave as ANTARCTIC_  

In [ ]:
# Create strings to access all the CryoSat files
years = np.arange(2011, 2022) 
yearsString = [str(year) for year in years]
months = np.arange(1, 13)
monthsString = [str(month).zfill(2) for month in months]
allTime = ['2010_08', '2010_09', '2010_10', '2010_11', '2010_12']+[year + "_" + month for year in yearsString for month in monthsString]

In [ ]:
# Import REMA Data
rema = rxr.open_rasterio(remaMosaicPath, masked=True).squeeze()

In [ ]:
def monthlyAnomaly(rema, date, long, lat):
    # Import CryoSat Data
    ds = xr.open_dataset(cryosatPath+date+"_V201.nc")
    elevation = ds['elevation']

    # Find Center of the Lake
    transformer = Transformer.from_crs(
        "EPSG:4326",   # lat/lon
        "EPSG:3031",   # Antarctic Polar Stereo
        always_xy=True
    )

    x_center, y_center = transformer.transform(long, lat)

    # Define Bounding Box Around Center of the Lake
    half_size = 50_000  # 50 km

    xmin = x_center - half_size
    xmax = x_center + half_size
    ymin = y_center - half_size
    ymax = y_center + half_size

    # Pull Boxes for CryoSat & REMA
    cryosat_small = elevation.sel(x=slice(xmin, xmax), y=slice(ymin, ymax))
    rema_small = rema.rio.clip_box(xmin,ymin,xmax,ymax)

    # Calculate Anomaly in Cryosat Based on REMA
    cryosat_small = cryosat_small.rio.write_crs("EPSG:3031")
    cryosat_small = cryosat_small.rio.set_spatial_dims(x_dim="x", y_dim="y")

    rema_small = rema_small.rio.set_spatial_dims(x_dim="x", y_dim="y")
    rema_matched = rema_small.rio.reproject_match(cryosat_small.isel(time=0))

    anomaly = cryosat_small.isel(time=0) - rema_matched

    return anomaly

In [ ]:
def locationAnomaly(long, lat, filePath, allTime):
    # gather anomaly arrays
    dataarrays = []
    for date in allTime:
        da = monthlyAnomaly(rema, date, long, lat)
        dataarrays.append(da)
    # Combine anomaly arrays & save to netCDF
    stacked = xr.concat(dataarrays, dim='time')
    ds = xr.Dataset({'anomaly': stacked})
    ds.to_netcdf('lakesTS/'+filePath+'.nc')


In [19]:
# create all your lake .netCDF files
# list generated by DeepSeek from Table 1 in Supplementary Information from Wilson et al. (2025)
locationAnomaly(110.09, -67.02, "Adams17", allTime)
locationAnomaly(109.65, -67.09, "Adams19", allTime)
locationAnomaly(66.28, -72.70, "Amery22", allTime)
locationAnomaly(109.42, -67.08, "ANZAC19", allTime)
locationAnomaly(168.32, -84.70, "Beardmore107", allTime)
locationAnomaly(171.67, -83.91, "Beardmore18", allTime)
locationAnomaly(171.33, -84.45, "Beardmore66", allTime)
locationAnomaly(51.64, -67.13, "Beaver27", allTime)
locationAnomaly(18.57, -72.60, "Borchgrevink188", allTime)
locationAnomaly(21.48, -72.57, "Borchgrevink198", allTime)
locationAnomaly(157.64, -80.51, "Byrd10", allTime)
locationAnomaly(153.13, -80.62, "Byrd97", allTime)
locationAnomaly(154.43, -81.13, "Byrd98", allTime)
locationAnomaly(153.91, -70.17, "CookEast134", allTime)
locationAnomaly(154.40, -70.48, "CookEast156", allTime)
locationAnomaly(149.29, -70.20, "CookWest186", allTime)
locationAnomaly(151.08, -69.19, "CookWest58", allTime)
locationAnomaly(150.20, -69.21, "CookWest67", allTime)
locationAnomaly(155.52, -74.43, "David180", allTime)
locationAnomaly(157.93, -75.17, "David80", allTime)
locationAnomaly(-70.58, -74.80, "Evans158", allTime)
locationAnomaly(61.43, -73.24, "Fisher168", allTime)
locationAnomaly(138.15, -66.96, "Francais36", allTime)
locationAnomaly(138.11, -67.04, "Francais46", allTime)
locationAnomaly(129.67, -67.29, "Frost28", allTime)
locationAnomaly(-78.64, -82.02, "Institute134", allTime)
locationAnomaly(-75.52, -81.08, "Institute14", allTime)
locationAnomaly(-75.27, -82.19, "Institute142", allTime)
locationAnomaly(-76.88, -82.44, "Institute171", allTime)
locationAnomaly(-76.86, -82.48, "Institute176", allTime)
locationAnomaly(-1.74, -72.51, "Jutulstraumen69", allTime)
locationAnomaly(-1.47, -72.54, "Jutulstraumen71", allTime)
locationAnomaly(-0.90, -72.67, "Jutulstraumen74", allTime)
locationAnomaly(26.48, -71.21, "KingBaudouinEast18", allTime)
locationAnomaly(28.94, -70.64, "KingBaudouinEast3", allTime)
locationAnomaly(29.90, -70.98, "KingBaudouinWest51", allTime)
locationAnomaly(74.18, -71.58, "Kronshtadskiy95", allTime)
locationAnomaly(-125.37, -75.65, "Kyoto96", allTime)
locationAnomaly(67.17, -74.40, "Lambert118", allTime)
locationAnomaly(69.38, -74.33, "Lambert119", allTime)
locationAnomaly(70.07, -74.45, "Lambert152", allTime)
locationAnomaly(67.67, -73.77, "Lambert45", allTime)
locationAnomaly(68.79, -74.07, "Lambert84", allTime)
locationAnomaly(67.25, -74.22, "Lambert99", allTime)
locationAnomaly(14.00, -70.68, "Lazarev35", allTime)
locationAnomaly(-131.59, -77.57, "MacAyeal299", allTime)
locationAnomaly(157.48, -69.86, "Matusevich53", allTime)
locationAnomaly(157.45, -69.89, "Matusevich57", allTime)
locationAnomaly(66.16, -74.33, "Mellor112", allTime)
locationAnomaly(63.33, -74.61, "Mellor183", allTime)
locationAnomaly(171.03, -85.58, "Mill161", allTime)
locationAnomaly(170.47, -85.61, "Mill165", allTime)
locationAnomaly(119.01, -67.56, "MoscowUniversity19", allTime)
locationAnomaly(117.01, -69.08, "MoscowUniversity194", allTime)
locationAnomaly(119.02, -67.74, "MoscowUniversity37", allTime)
locationAnomaly(118.07, -67.67, "MoscowUniversity44", allTime)
locationAnomaly(119.47, -67.91, "MoscowUniversity60", allTime)
locationAnomaly(117.99, -67.79, "MoscowUniversity62", allTime)
locationAnomaly(158.49, -78.69, "Mulock60", allTime)
locationAnomaly(146.19, -69.86, "Ninnis160", allTime)
locationAnomaly(148.20, -68.58, "NinnisEast10", allTime)
locationAnomaly(148.32, -69.06, "NinnisEast63", allTime)
locationAnomaly(-33.34, -81.09, "Recovery50", allTime)
locationAnomaly(-30.50, -81.03, "Recovery68", allTime)
locationAnomaly(-89.14, -76.99, "Rutford192", allTime)
locationAnomaly(-152.72, -85.56, "Scott12", allTime)
locationAnomaly(-152.92, -85.64, "Scott19", allTime)
locationAnomaly(-120.03, -75.36, "Stockholm72", allTime)
locationAnomaly(-120.02, -75.39, "Stockholm77", allTime)
locationAnomaly(-47.65, -84.02, "SupportForce153", allTime)
locationAnomaly(-46.41, -82.60, "SupportForce3", allTime)
locationAnomaly(112.49, -69.15, "Totten191", allTime)
locationAnomaly(113.79, -67.82, "Totten35", allTime)
locationAnomaly(112.89, -67.60, "Totten36", allTime)
locationAnomaly(113.30, -67.77, "Totten40", allTime)
locationAnomaly(112.62, -67.58, "Totten52", allTime)
locationAnomaly(113.89, -67.58, "Totten7", allTime)
locationAnomaly(112.21, -67.91, "Totten82", allTime)
locationAnomaly(110.70, -66.82, "Vanderford12", allTime)
locationAnomaly(110.77, -66.72, "Vanderford4", allTime)
locationAnomaly(110.73, -66.76, "Vanderford5", allTime)
locationAnomaly(7.92, -70.56, "Vigrid4", allTime)
locationAnomaly(-140.06, -83.53, "Whillans180", allTime)
locationAnomaly(-59.07, -84.22, "Academy45", allTime)
locationAnomaly(-55.70, -84.34, "Academy91", allTime)

In [24]:
# data visuallization to sanity check 

def makeGIF(fileName): 
    ds = xr.open_dataset("lakesTS/"+fileName+".nc")
    anomaly = ds['anomaly']
    cmap="RdBu_r"
    fig, ax = plt.subplots()
    def update(frame):
        ax.cla()
        im = anomaly.isel(time=frame).plot(ax=ax, cmap=cmap, vmin=-5, vmax=5, add_colorbar=False)
        ax.set_title(f"{fileName} - Time Step {frame}")
        ax.set_xlabel("")
        ax.set_ylabel("")

    # Add a single colorbar
    im = anomaly.isel(time=0).plot(ax=ax, cmap=cmap, vmin=-5, vmax=5, add_colorbar=False)
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("Elevation Anomaly (m)")

    anim = FuncAnimation(fig, update, frames=len(anomaly.time))
    anim.save("gifs/anim"+fileName+".gif", writer='imagemagick')
    plt.close()


In [25]:
makeGIF("Adams17")
makeGIF("Adams19")
makeGIF("Amery22")
makeGIF("ANZAC19")
makeGIF("Beardmore107")
makeGIF("Beardmore18")
makeGIF("Beardmore66")
makeGIF("Beaver27")
makeGIF("Borchgrevink188")
makeGIF("Borchgrevink198")
makeGIF("Byrd10")
makeGIF("Byrd97")
makeGIF("Byrd98")
makeGIF("CookEast134")
makeGIF("CookEast156")
makeGIF("CookWest186")
makeGIF("CookWest58")
makeGIF("CookWest67")
makeGIF("David180")
makeGIF("David80")
makeGIF("Evans158")
makeGIF("Fisher168")
makeGIF("Francais36")
makeGIF("Francais46")
makeGIF("Frost28")
makeGIF("Institute134")
makeGIF("Institute14")
makeGIF("Institute142")
makeGIF("Institute171")
makeGIF("Institute176")
makeGIF("Jutulstraumen69")
makeGIF("Jutulstraumen71")
makeGIF("Jutulstraumen74")
makeGIF("KingBaudouinEast18")
makeGIF("KingBaudouinEast3")
makeGIF("KingBaudouinWest51")
makeGIF("Kronshtadskiy95")
makeGIF("Kyoto96")
makeGIF("Lambert118")
makeGIF("Lambert119")
makeGIF("Lambert152")
makeGIF("Lambert45")
makeGIF("Lambert84")
makeGIF("Lambert99")
makeGIF("Lazarev35")
makeGIF("MacAyeal299")
makeGIF("Matusevich53")
makeGIF("Matusevich57")
makeGIF("Mellor112")
makeGIF("Mellor183")
makeGIF("Mill161")
makeGIF("Mill165")
makeGIF("MoscowUniversity19")
makeGIF("MoscowUniversity194")
makeGIF("MoscowUniversity37")
makeGIF("MoscowUniversity44")
makeGIF("MoscowUniversity60")
makeGIF("MoscowUniversity62")
makeGIF("Mulock60")
makeGIF("Ninnis160")
makeGIF("NinnisEast10")
makeGIF("NinnisEast63")
makeGIF("Recovery50")
makeGIF("Recovery68")
makeGIF("Rutford192")
makeGIF("Scott12")
makeGIF("Scott19")
makeGIF("Stockholm72")
makeGIF("Stockholm77")
makeGIF("SupportForce153")
makeGIF("SupportForce3")
makeGIF("Totten191")
makeGIF("Totten35")
makeGIF("Totten36")
makeGIF("Totten40")
makeGIF("Totten52")
makeGIF("Totten7")
makeGIF("Totten82")
makeGIF("Vanderford12")
makeGIF("Vanderford4")
makeGIF("Vanderford5")
makeGIF("Vigrid4")
makeGIF("Whillans180")
makeGIF("Academy45")
makeGIF("Academy91")

MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instea